# NE PAS COMMITER CE NOTEBOOK AVEC LES MOTS DE PASSE EN CLAIR

In [1]:
%pylab inline
from common import RobotCamomile
from getpass import getpass
from tortilla.utils import bunchify
import numpy as np
from datetime import datetime

Populating the interactive namespace from numpy and matplotlib


In [2]:
client = RobotCamomile('http://api.mediaeval.niderb.fr', 'robot_label')

Password for robot_label: ········


In [3]:
corpus = client.getCorpusByName('mediaeval.test')
allLayer = client.getLayerByName(corpus, 'mediaeval.groundtruth.label.all')
consensusLayer = client.getLayerByName(corpus, 'mediaeval.groundtruth.label.consensus')

#### Load all annotations

In [4]:
allAnnotations = []
for medium, annotations in client.getAnnotations_iter(allLayer):
    allAnnotations += annotations

#### Load consensus

In [5]:
consensusAnnotations = client.getAnnotations(consensusLayer)

In [6]:
consensus = {c.fragment: bunchify(c.data) for c in consensusAnnotations}

#### Stats

In [7]:
# total number of annotations, per annotator
count = {}
# durations of each annotation, per annotator
duration = {}
# date of each annotation, per annotation
date = {}
# total number of annotations for which a consensus was reached, per annotator
countWithConsensus = {}
# total number of annotations for which dontKnow was selected at least once, per annotator
dontKnow = {}
# total number of annotations that differ from consensus, per annotator
bad = {}

for annotation in allAnnotations:
    
    # skip old annotations
    if 'log' not in annotation.data:
        continue
    
    # which shot was annotated?
    shot = annotation.fragment
    
    # who annotated this shot?
    annotator = annotation.data.annotator
    
    # how long did it take to produre this annotations?
    duration.setdefault(annotator, []).append(annotation.data.log.duration)

    # it may happen that
    if 'date' in annotation.data.log:
        date.setdefault(annotator, []).append(datetime.strptime(annotation.data.log.date, '%Y-%m-%dT%H:%M:%S.%fZ'))

    # number of annotations by this annotator
    count[annotator] = count.get(annotator, 0) + 1

    status = annotation.data.get('known', {})
    status['?unknown?'] = 'speakingFace' if annotation.data.unknown else 'noFace'

    if 'dontKnow' in status.values():
        dontKnow[annotator] = dontKnow.get(annotator, 0) + 1

    # number of annotations by this annotator for which a consensus was reached
    if shot not in consensus:
        continue

    countWithConsensus[annotator] = countWithConsensus.get(annotator, 0) + 1
    k = consensus[shot]
    if k == '':
        k = {}
    
    kSpeakingFace = set([personName for personName, decision in k.iteritems() if decision == 'speakingFace'])
    aSpeakingFace = set([personName for personName, decision in status.iteritems() if decision == 'speakingFace'])
    
    if kSpeakingFace != aSpeakingFace:
        missed = kSpeakingFace - aSpeakingFace
        falseAlarm = aSpeakingFace - kSpeakingFace
        bad[annotator] = bad.get(annotator, 0) + 1        

In [8]:
def timeline(date, annotator):
    allDates = [date2num(d) for d in date[annotator]]
    m, M = int(floor(min(allDates))), int(ceil(max(allDates)))
    nDays = M+1-m
    figsize(nDays*5, 2)
    hist(allDates, bins=linspace(m, M+1, 24*(M-m+1)))
    xlim(m, M)
    xticks(range(m, M+1), [datetime.strftime(num2date(d), '%Y/%m/%d') for d in range(m, M+1)]);
    title('{annotator} - {n} annotations'.format(annotator=annotator, n=len(date[annotator])))
    savefig('/tmp/{annotator}.png'.format(annotator=annotator))
    close()

In [9]:
for annotator in count:
    totalNumberOfAnnotations = count[annotator]
    estimatedAccuracy = 100. - 100. * bad.get(annotator, 0) / countWithConsensus[annotator]
    dontKnowRatio = 100. * dontKnow.get(annotator, 0) / count[annotator]
    print annotator
    print totalNumberOfAnnotations, 'shots'
    print 'Quality {accuracy:.2f}%'.format(accuracy=estimatedAccuracy)
    print 'Skipped {dontKnow:.1f}%'.format(dontKnow=dontKnowRatio)
    timeline(date, annotator)
    print

herve.bredin
618 shots
Quality 99.03%
Skipped 0.6%

bredin
1415 shots
Quality 97.24%
Skipped 0.6%

barras
1620 shots
Quality 97.50%
Skipped 0.2%

paula.lopez
240 shots
Quality 100.00%
Skipped 1.7%

johann.poignant
3342 shots
Quality 99.52%
Skipped 0.2%

di.wu
129 shots
Quality 100.00%
Skipped 3.1%

poignant
5112 shots
Quality 99.08%
Skipped 0.1%

fake1
291 shots
Quality 99.31%
Skipped 0.0%

rosalia.barros
192 shots
Quality 98.16%
Skipped 6.2%

chomicha.bendahman
1053 shots
Quality 99.72%
Skipped 1.2%

quentin.samier
806 shots
Quality 100.00%
Skipped 0.2%

amina.taoudi
5010 shots
Quality 99.69%
Skipped 1.2%

